# Redes Neuronales - TP2
## Ej 3

Implemente un perceptrón multicapa que aprenda la función lógica XOR de 2 y de 4
entradas (utilizando el algoritmo Backpropagation y actualizando en batch). Muestre
cómo evoluciona el error durante el entrenamiento.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [4]:
class perceptron_multicapa:
    def __init__(self, capas, dim_entrada):
        """
        capas: lista con la cantidad de perceptrones por capa (ej: [3,2,1])
        dim_entrada: tamaño del vector de entrada (sin bias)
        """
        self.capas = capas
        self.dim_entrada = dim_entrada
        self.lista_matrices = []
        self.lr = None
        
        # Inicializar matrices de pesos para cada capa
        entrada_anterior = dim_entrada
        for num_perceptrones in capas:
            # Cada matriz tiene shape (num_perceptrones, entrada_anterior + 1) (+1 por bias)
            matriz_pesos = np.random.uniform(-1, 1, size=(num_perceptrones, entrada_anterior + 1)) # arranca con pesos aleatorios
            self.lista_matrices.append(matriz_pesos)
            entrada_anterior = num_perceptrones

    def funcion_activacion(self, x):
        return 1 / (1 + np.exp(-x)) # función de activación tipo sigmoide
    
    def forward(self, x):
        """
        Propagación hacia adelante
        x: vector de entrada (sin bias)
        """
        a = np.concatenate(([1], x))  # Añadir bias a la entrada
        for W in self.lista_matrices:  # recorremos todas las matrices de pesos
            z = np.dot(W, a)
            a = self.funcion_activacion(z)
            a = np.concatenate(([1], a))  # Añadir bias para la siguiente capa
        return a[1:]  # Devolver salida sin bias
    
    def predecir(self, X):
        """
        X: matriz de entradas (cada fila es un vector de entrada)
        """
        return np.array([self.forward(x) for x in X])
    
    def entrenar(self, X, Y, lr=0.01, epochs=1000):
        """
        Entrenamiento del perceptrón multicapa usando backpropagation
        X: matriz de entradas (cada fila es un vector de entrada)
        Y: matriz de salidas esperadas (cada fila es un vector de salida)
        lr: tasa de aprendizaje
        epochs: cantidad de iteraciones sobre el conjunto de datos
        """
        self.lr = lr
        n_samples = X.shape[0]
        
        for epoch in range(epochs):
            for i in range(n_samples):
                # tomamos las 
                x = X[i]
                y = Y[i]
                
                # Forward pass
                activations = [np.concatenate(([1], x))]  # Lista de activaciones por capa (con bias)
                for W in self.lista_matrices:
                    z = np.dot(W, activations[-1])
                    a = self.funcion_activacion(z)
                    activations.append(np.concatenate(([1], a)))  # Añadir bias
                
                # Backward pass
                delta = activations[-1][1:] - y  # Error en la salida (sin bias)
                for layer in reversed(range(len(self.lista_matrices))):
                    a_prev = activations[layer]
                    W = self.lista_matrices[layer]
                    
                    # Gradiente
                    grad = np.outer(delta, a_prev)
                    
                    # Actualizar pesos
                    self.lista_matrices[layer] -= self.lr * grad
                    
                    if layer > 0:
                        # Calcular delta para la capa anterior (sin bias)
                        delta = np.dot(W[:, 1:].T, delta) * activations[layer][1:] * (1 - activations[layer][1:])
            
            if epoch % 100 == 0:
                loss = np.mean((self.predecir(X) - Y) ** 2)
                print(f"Epoch {epoch}, Loss: {loss}")
    

In [5]:
# datos para la XOR de 2 entradas y 1 salida
A = np.array([-1,1,-1,1])
B = np.array([-1,-1,1,1])
Y1= np.array([-1,1,-1,1])
datos_XOR2 = np.column_stack((A, B, Y1))


In [6]:
A = np.array([-1,1,-1,1,-1,1,-1,1,-1,1,-1,1,-1,1,-1,1])
B = np.array([-1,-1,1,1,-1,-1,1,1,-1,-1,1,1,-1,-1,1,1])
C = np.array([-1,-1,-1,-1,1,1,1,1,-1,-1,-1,-1,1,1,1,1])
D = np.array([-1,-1,-1,-1,-1,-1,-1,-1,1,1,1,1,1,1,1,1])
Y1 = np.where(np.logical_xor.reduce([A == 1, B == 1, C == 1, D == 1]), 1, -1)
datos_XOR4 = np.column_stack((A, B, C, D, Y1))


In [ ]:
# ahora toca entrenar el perceptron con estos datos
test2 = perceptron_multicapa(capas=[2, 1], dim_entrada=2) # para la XOR de 2 entradas y 1 salida
test4 = perceptron_multicapa(capas=[4, 1], dim_entrada=4) # para la XOR de 4 entradas y 1 salida


Epoch 0, Loss: 1.4589204984767417
Epoch 100, Loss: 0.6130755149426634
Epoch 200, Loss: 0.612770292018128
Epoch 300, Loss: 0.6127963972311867


Epoch 400, Loss: 0.6128074705392885
Epoch 500, Loss: 0.6128127950024782
Epoch 600, Loss: 0.6128157202559077
Epoch 700, Loss: 0.6128174905894604
Epoch 800, Loss: 0.6128186407184384
Epoch 900, Loss: 0.6128194291699646
Epoch 700, Loss: 0.6128174905894604
Epoch 800, Loss: 0.6128186407184384
Epoch 900, Loss: 0.6128194291699646
Predicciones para XOR 2 entradas:
Predicciones para XOR 2 entradas:
